In [1]:
from bayescache.data import Anon
from bayescache.data import train_valid_split

In [2]:
train = Anon('/Users/yngtodd/data', partition='train', download=True)

In [3]:
train

Dataset Anon
    Number of datapoints: 2500
    Split: train
    Root Location: /Users/yngtodd/data

In [17]:
train, valid = train_valid_split(train)

In [18]:
len(train)

2000

In [19]:
len(valid)

500